In [11]:
import numpy as np

from utils import *

import xgboost as xgb
from boostaroota import BoostARoota

In [2]:
train = load_data('../data/assignment_2_train_I.csv')
train[:2]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180000 entries, 0 to 179999
Columns: 457 entries, TransactionID to log_TransactionAmt
dtypes: float64(443), int64(9), object(5)
memory usage: 627.6+ MB


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,addr2_D15_std,card1_card2_D15_mean,card1_card2_D15_std,card1_card2_card3_card5_D15_mean,card1_card2_card3_card5_D15_std,card1_card2_card3_card5_addr1_addr2_D15_mean,card1_card2_card3_card5_addr1_addr2_D15_std,int_TransactionAmt,frac_TransactionAmt,log_TransactionAmt
0,2987000,0,2017-12-02 00:00:00,68.5,0,13926,NaN,150.0,3.0,142.0,...,186.33749,0.500000,0.577350,0.500000,0.577350,0.00,NaN,68,0.5,4.226834
1,2987001,0,2017-12-02 00:00:01,29.0,0,2755,404.0,150.0,1.0,102.0,...,186.33749,114.811765,177.751006,114.811765,177.751006,59.75,123.056897,29,0.0,3.367296


In [3]:
trash_cols = [col for col, val in (train.dtypes == 'object').items() if val]

train.drop(trash_cols, axis=1, inplace=True)
train.shape

(180000, 452)

# Задание 4: использовать BoostARoota для отбора признаков (настраивать или нет - на ваше усмотрение), переобучить модель и оценить изменение качества.

In [4]:
Xtrain, Xvalid, ytrain, yvalid = train_test_split(
    train.drop('isFraud', axis=1), train['isFraud'],
    train_size=0.8, random_state=42, shuffle=True
)

In [5]:
%%time
br = BoostARoota(metric='auc', silent=True)
br.fit(Xtrain, ytrain)

[23:50:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:50:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:51:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

[23:57:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:58:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:58:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

In [6]:
br.keep_vars_

0                                     TransactionID
1                                    TransactionAmt
2                                         ProductCD
3                                             card1
4                                             card2
                           ...                     
238                             card1_card2_D15_std
239                card1_card2_card3_card5_D15_mean
240    card1_card2_card3_card5_addr1_addr2_D15_mean
241     card1_card2_card3_card5_addr1_addr2_D15_std
242                             frac_TransactionAmt
Name: feature, Length: 236, dtype: object

In [8]:
Xtrain_br = br.transform(Xtrain)
Xvalid_br = br.transform(Xvalid)

dtrain_br = xgb.DMatrix(Xtrain_br, label=ytrain)
dvalid_br = xgb.DMatrix(Xvalid_br, label=yvalid)

In [14]:
%%time
params = {
    "eta": 0.01,
    "objective": "binary:logistic",
    "subsample": 0.5,
    "base_score": np.mean(ytrain),
    "eval_metric": "auc",
    'tree_method': 'gpu_hist'
}

xgb_model_br = xgb.train(
    params=params,
    dtrain=dtrain_br,
    num_boost_round=1000,
    evals=[(dtrain_br, "train"), (dvalid_br, "valid")],
    early_stopping_rounds=50,
    verbose_eval=100
)

[0]	train-auc:0.81039	valid-auc:0.79881
[100]	train-auc:0.89777	valid-auc:0.88492
[200]	train-auc:0.92177	valid-auc:0.90516
[300]	train-auc:0.93724	valid-auc:0.91738
[400]	train-auc:0.94634	valid-auc:0.92478
[500]	train-auc:0.95350	valid-auc:0.92939
[600]	train-auc:0.95928	valid-auc:0.93335
[700]	train-auc:0.96372	valid-auc:0.93654
[800]	train-auc:0.96783	valid-auc:0.93971
[900]	train-auc:0.97147	valid-auc:0.94191
[999]	train-auc:0.97437	valid-auc:0.94407
Wall time: 52.1 s


In [15]:
scores = ScoresAUC('roc_auc')
scores.load_csv()
scores.add_score(round(xgb_model_br.best_score, 4), 'boostaroota')

,roc_auc
base_line,0.9384
above_zero,0.9383
perm_imp,0.9389
perm_imp_zero,0.9363
boostaroota,0.9441


## Вывод:
- BootsARoota показал больший скор, чем предыдущие варианты